Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [128]:
data = pd.read_csv("umn_foursquare_datasets/checkins.dat", sep="|")

In [129]:
data.columns = [x.strip("  ") for x in data.columns]

In [130]:
data = data[~data.latitude.isnull()]
data = data[~data.longitude.isnull()]
data = data[~(data.latitude=='                   ')]
data = data[~(data.longitude=='                   ')]

In [131]:
data.shape

(396634, 6)

In [136]:
data_clean = data.drop_duplicates(subset=["latitude", "longitude"])

In [137]:
data_clean.head()

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.4483771,-112.0740373,2012-04-21 17:38:18


In [139]:
data_clean.shape

(11798, 6)

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [36]:
data = data.loc[:100000]

In [140]:
from sklearn.cluster import MeanShift

In [141]:
clust = MeanShift(bandwidth=0.1, min_bin_freq=15, n_jobs=-1)
clust.fit(data_clean[["latitude", "longitude"]])

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=15,
     n_jobs=-1, seeds=None)

In [142]:
val, counts = np.unique(clust.labels_, return_counts=True)

In [143]:
clust.labels_.shape

(11798,)

In [144]:
val[counts>15]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  23,  24,  25,  30,  31,
        32,  33,  34,  35,  36,  37,  39,  40,  41,  42,  48,  54,  55,
        56,  57,  58,  61,  68,  71,  72,  74,  75,  94, 114, 121, 127, 143])

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров.

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

- 33.751277, -118.188740 (Los Angeles)

- 25.867736, -80.324116 (Miami)

- 51.503016, -0.075479 (London)

- 52.378894, 4.885084 (Amsterdam)

- 39.366487, 117.036146 (Beijing)

- -33.868457, 151.205134 (Sydney)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [145]:
cities = {
    "Los Angeles": [33.751277, -118.188740],
    "Miami":[25.867736, -80.324116],
    "London":[51.503016, -0.075479],
    "Amsterdam":[52.378894, 4.885084],
    "Beijing":[39.366487, 117.036146],
    "Sydney":[-33.868457, 151.205134]
}

In [146]:
ind

array([332, 261,  34, 294, 195, 256, 245,  67, 296, 338, 212,  65, 246,
        94, 340, 206, 259,  12, 274, 218])

In [157]:
closest = {}

centers = clust.cluster_centers_[counts>15]

for city, coords in cities.items():    
    dist = ((centers - coords)**2).sum(axis=1)**0.5
    ind = (dist).argsort()[:20]
    closest[city] = [dist[ind], clust.cluster_centers_[ind]]

In [158]:
minimal_dist = np.inf

for city, value in closest.items():
    for dist in value[0]:
        if dist < minimal_dist:
            minimal_dist = dist
            answer = value[1][np.where(value[0]==dist)]

In [159]:
closest

{'Amsterdam': [array([  5.10033077,  76.64181683,  79.28080923,  79.45457813,
          79.52081831,  79.57554047,  79.68027296,  79.71331282,
          79.75914979,  79.83998088,  79.85097807,  79.85177647,
          79.8825977 ,  80.03313752,  80.04021236,  80.08089184,
          80.14644039,  80.21856099,  80.3589498 ,  80.89713376]),
  array([[  5.15166422e+01,  -1.41833124e-01],
         [  4.23390451e+01,  -7.10962921e+01],
         [  4.06428014e+01,  -7.42900366e+01],
         [  4.07196150e+01,  -7.37093891e+01],
         [  4.04322605e+01,  -7.98653569e+01],
         [  4.07232198e+01,  -7.38322070e+01],
         [  3.99637223e+01,  -7.52825240e+01],
         [  4.07447922e+01,  -7.39746644e+01],
         [  4.08551350e+01,  -7.40371876e+01],
         [  4.05503068e+01,  -7.44565952e+01],
         [  2.59266495e+01,  -8.01785137e+01],
         [ -3.38796290e+01,   1.51199013e+02],
         [  4.08287568e+01,  -7.41580925e+01],
         [  1.45817493e+01,   1.21034150e+02],
  

In [160]:
minimal_dist

0.012739094406399388

In [164]:
with open("answer.txt", "w") as f:
    f.write(str([-33.860630428571433, 151.20477592857145]))

In [165]:
answer

array([[ 40.7199641, -73.5337662]])